In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.contrib import slim
import os
import tarfile
from six.moves import urllib

In [ ]:
from tensorflow.contrib.slim.nets import vgg

## Imagenet index to label name

In [ ]:
def create_readable_names_for_imagenet_labels():
    """Create a dict mapping label id to human readable string.
    Returns:
    labels_to_names: dictionary where keys are integers from to 1000
    and values are human-readable names.
    We retrieve a synset file, which contains a list of valid synset labels used
    by ILSVRC competition. There is one synset one per line, eg.
    #   n01440764
    #   n01443537
    We also retrieve a synset_to_human_file, which contains a mapping from synsets
    to human-readable names for every synset in Imagenet. These are stored in a
    tsv format, as follows:
    #   n02119247    black fox
    #   n02119359    silver fox
    We assign each synset (in alphabetical order) an integer, starting from 1
    (since 0 is reserved for the background class).
    Code is based on
    https://github.com/tensorflow/models/blob/master/research/inception/inception/data/build_imagenet_data.py#L463
    """

    # pylint: disable=g-line-too-long
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/inception/inception/data/'
    synset_url = '{}/imagenet_lsvrc_2015_synsets.txt'.format(base_url)
    synset_to_human_url = '{}/imagenet_metadata.txt'.format(base_url)

    filename, _ = urllib.request.urlretrieve(synset_url)
    synset_list = [s.strip() for s in open(filename).readlines()]
    num_synsets_in_ilsvrc = len(synset_list)
    assert num_synsets_in_ilsvrc == 1000

    filename, _ = urllib.request.urlretrieve(synset_to_human_url)
    synset_to_human_list = open(filename).readlines()
    num_synsets_in_all_imagenet = len(synset_to_human_list)
    assert num_synsets_in_all_imagenet == 21842

    synset_to_human = {}
    for s in synset_to_human_list:
        parts = s.strip().split('\t')
        assert len(parts) == 2
        synset = parts[0]
        human = parts[1]
        synset_to_human[synset] = human

    label_index = 1
    labels_to_names = {0: 'background'}
    for synset in synset_list:
        name = synset_to_human[synset]
        labels_to_names[label_index] = name
        label_index += 1

    return labels_to_names

## Download Pretrained Model's Checkpoint

In [ ]:
ckpt_dir = "ckpt"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [ ]:
ckpt_url = "http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz"
zpath = os.path.join(ckpt_dir, "vgg_16_2016_08_28.tar.gz")
if not os.path.exists(zpath):
    print ("Downloading %s ..." % (zpath))
    urllib.request.urlretrieve(ckpt_url, zpath)
    print ("Done!")
else:
    print ("%s Already Exists" % (zpath))

## Unzip Checkpoint

In [ ]:
cpath = os.path.join(ckpt_dir, "vgg_16_2016_08_28")
if not os.path.exists(cpath):
    print ("Extracting %s ..." % (cpath))
    tar = tarfile.open(zpath, "r:gz")
    tar.extractall(path=cpath)
    tar.close()
    print ("Done!")
else:
    print ("%s Already Exists" % (cpath))

## Model

In [ ]:
x = tf.placeholder(shape=[None, 224, 224, 3], dtype=tf.float32)

with slim.arg_scope(vgg.vgg_arg_scope()):
    mean = tf.constant([123.68, 116.78, 103.94], dtype=tf.float32, shape=[1,1,1,3])
    im_centered = x - mean
    logits, end_points = vgg.vgg_16(inputs=im_centered, num_classes=1000)
    preds = tf.nn.softmax(logits, axis=-1)

## Restore

In [ ]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))

In [ ]:
ckpt_name = "vgg_16.ckpt"
ckpt_path = os.path.join(cpath, ckpt_name)

variables_to_restore = slim.get_variables_to_restore()
saver = tf.train.Saver(variables_to_restore)

saver.restore(sess, ckpt_path)

## Inference - Single Image

In [ ]:
from PIL import Image

currimg = Image.open("data/boat.jpg")
#currimg = Image.open("data/dog.jpg")
currimg = currimg.resize((224, 224))
image = np.asarray(currimg)
image = np.reshape(image, (-1, 224, 224, 3))
image.shape

In [ ]:
plt.imshow(image[0])
plt.show()

In [ ]:
predict = sess.run(preds, feed_dict={x:image})
predict.shape

In [ ]:
pred_class = np.argmax(predict, axis=-1)
pred_class

In [ ]:
index = int(pred_class)

In [ ]:
predict[0,index]

In [ ]:
name = create_readable_names_for_imagenet_labels()

In [ ]:
name[index+1]

In [ ]:
k = 5
top_k = np.argsort(predict, axis=-1)[0, -k:][::-1]
print (top_k)

In [ ]:
for index in top_k:
    print (name[index+1], predict[0,index])    